# Creando dataset

In [1]:
import pandas as pd 
import numpy as np 
import json

Deteccion de microparadas
Flujo:
1. Primero generaremos csv de microparadas para cada objeto
2. Como seran csv entonces ya 

In [2]:
df_1 = pd.read_csv("../BD_Innovathon/Objeto_1_ES_pos.csv")
df_2 = pd.read_csv("../BD_Innovathon/Objeto_2_ES_pos.csv")
print(df_2.head(3))

   Id         Fecha y hora               posición  Distancia (m)  \
0   1  29/09/2025 22:15:14  S12 04.069 W77 03.038              0   
1   2  29/09/2025 22:16:14  S12 04.134 W77 02.957            191   
2   3  29/09/2025 22:16:44  S12 04.137 W77 02.956              5   

  Tiempo de registro (s)  Velocidad (km/h)  
0                0:00:00                 0  
1                0:01:00                11  
2                0:00:30                 1  


In [10]:
def detect_microparadas_by_object(dataframe_object, object_id):
    # for para cada registro    
    dataframe_object['is_microparada'] = dataframe_object['Velocidad (km/h)'].apply(lambda x: 1 if x < 6 else 0)
    ultima_fila = dataframe_object.iloc[-1]
    alerta = bool(ultima_fila['is_microparada'] == 1)

    # Respuesta tipo JSON para la mini app
    result = {
        "objeto_id": object_id,
        "alerta": alerta,
        "fecha_hora": str(ultima_fila['Fecha y hora']),
        "posicion": ultima_fila['posición'],
        "velocidad": ultima_fila['Velocidad (km/h)']
    }
    return result
    

In [11]:
dft_1 = detect_microparadas_by_object(df_1, 1)
print(dft_1)
#dft_2 = detect_microparadas_by_object(df_2, objeto_id=2) 
#print(dft_2.shape)
#dft_2 = detect_microparadas_by_object(df_2, objeto_id=2)
#dft_3 = detect_microparadas_by_object(df_3, objeto_id=3)

{'objeto_id': 1, 'alerta': True, 'fecha_hora': '29/09/2025 23:19:19', 'posicion': 'S11 59.972 W76 49.870', 'velocidad': np.int64(0)}


In [13]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 1. Cargar datos y detectar microparadas
df = pd.read_csv("../BD_Innovathon/Objeto_1_ES_pos.csv")
df['is_microparada'] = df['Velocidad (km/h)'].apply(lambda v: 1 if v < 6 else 0)
df['hora'] = pd.to_datetime(df['Fecha y hora'], dayfirst=True).dt.hour
df['dia_semana'] = pd.to_datetime(df['Fecha y hora'], dayfirst=True).dt.dayofweek


# 2. Generar etiquetas (congestión si hay muchas microparadas en un lugar/hora)
trafico = df.groupby(['posición','hora']).agg({'is_microparada':'sum'}).reset_index()
trafico['y'] = (trafico['is_microparada'] >= 3).astype(int)

# 3. Variables predictoras
X = trafico[['hora']]
y = trafico['y']

# 4. Modelo simple
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# 5. Predicción
predicciones = modelo.predict(X_test)



In [18]:
print(predicciones)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
# Analizar los datos para entender mejor el problema
print("Análisis de datos:")
print(f"Total de microparadas: {df['is_microparada'].sum()}")
print(f"Porcentaje de microparadas: {df['is_microparada'].mean()*100:.2f}%")
print("\nDistribución por hora:")
print(df.groupby('hora')['is_microparada'].agg(['count', 'sum', 'mean']))

print("\nDatos de tráfico agrupados:")
print(trafico.head(10))
print(f"\nEtiquetas de congestión (y=1): {trafico['y'].sum()}")
print(f"Total de observaciones: {len(trafico)}")

Análisis de datos:
Total de microparadas: 25
Porcentaje de microparadas: 30.12%

Distribución por hora:
      count  sum      mean
hora                      
22       62   18  0.290323
23       21    7  0.333333

Datos de tráfico agrupados:
                posición  hora  is_microparada  y
0  S11 59.937 W76 50.025    23               0  0
1  S11 59.938 W76 50.023    23               1  0
2  S11 59.938 W76 50.028    23               0  0
3  S11 59.972 W76 49.870    23               3  1
4  S11 59.972 W76 49.872    23               3  1
5  S11 59.974 W76 49.878    23               0  0
6  S11 59.975 W76 49.869    23               0  0
7  S12 00.021 W76 49.986    23               0  0
8  S12 00.021 W76 49.989    23               0  0
9  S12 00.029 W76 49.994    23               0  0

Etiquetas de congestión (y=1): 3
Total de observaciones: 73


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import argparse
import re
import sys
from datetime import datetime, timedelta
import pandas as pd

# === Helpers ===

POS_RE = re.compile(
    r'^\s*([NS])\s*(\d{1,2})\s+(\d{1,2}\.\d+)\s+([EWOW])\s*(\d{1,3})\s+(\d{1,2}\.\d+)\s*$',
    re.IGNORECASE
)
# Nota: permitimos 'W' normal; el grupo [EWOW] incluye una 'O' por si viniera "Oeste" (O).
# Si tu data siempre es W, igual funciona.

def pos_sw_to_latlon(pos_str: str):
    """
    Convierte 'S12 01.393 W76 54.690' → (lat, lon) en decimales.
    lat = -(12 + 1.393/60) por 'S'
    lon = -(76 + 54.690/60) por 'W'
    """
    if pd.isna(pos_str):
        return (None, None)
    s = str(pos_str).strip()
    m = POS_RE.match(s)
    if not m:
        # Intento alternativo: reemplazar 'O' por 'W' y reintentar
        s2 = s.replace(' O', ' W').replace(' o', ' w')
        m = POS_RE.match(s2)
        if not m:
            return (None, None)

    hemi_lat, deg_lat, min_lat, hemi_lon, deg_lon, min_lon = m.groups()
    deg_lat = float(deg_lat); min_lat = float(min_lat)
    deg_lon = float(deg_lon); min_lon = float(min_lon)

    lat = deg_lat + (min_lat / 60.0)
    lon = deg_lon + (min_lon / 60.0)

    hemi_lat = hemi_lat.upper()
    hemi_lon = hemi_lon.upper().replace('O', 'W')  # tratar 'O' (Oeste) como 'W'







    if hemi_lat == 'S':
        lat = -lat
    elif hemi_lat == 'N':
        lat = +lat
    else:
        return (None, None)

    if hemi_lon == 'W':
        lon = -lon
    elif hemi_lon == 'E':
        lon = +lon
    else:
        return (None, None)

    return (round(lat, 9), round(lon, 9))  # 9 decimales ≈ precisión sub-métrica

def parse_datetime_peru(dt_str: str):
    """
    Convierte '29/09/2025 22:15:02' → (año, mes, día, hora, minuto, segundo)
    Formato esperado: DD/MM/YYYY HH:MM:SS
    """
    if pd.isna(dt_str):
        return (None,)*6
    s = str(dt_str).strip()
    try:
        dt = datetime.strptime(s, "%d/%m/%Y %H:%M:%S")
        return dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second
    except ValueError:
        # fallback común: si hubiera ceros a la izquierda omitidos
        try:
            dt = pd.to_datetime(s, dayfirst=True, errors='raise')
            return dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second
        except Exception:
            return (None,)*6

def hms_to_seconds(hms_str: str):
    """
    Convierte 'H:MM:SS' (p.ej. '0:01:00') → 60 (segundos).
    Si es '0' o vacío → 0.
    """
    if pd.isna(hms_str):
        return 0
    s = str(hms_str).strip()
    if s in ("", "0", "00:00:00", "0:00:00"):
        return 0
    parts = s.split(':')
    try:
        if len(parts) == 3:
            h, m, sec = int(parts[0]), int(parts[1]), int(round(float(parts[2])))
            return h*3600 + m*60 + sec
        elif len(parts) == 2:  # MM:SS
            m, sec = int(parts[0]), int(round(float(parts[1])))
            return m*60 + sec
        elif len(parts) == 1:  # Solo segundos
            return int(round(float(parts[0])))
    except Exception:
        pass
    # Último recurso: intentar con pandas
    try:
        td = pd.to_timedelta(s)
        return int(td.total_seconds())
    except Exception:
        return 0

# === Main ===

def main():
    ap = argparse.ArgumentParser(description="Prepara micro-paradas: lat/lon, descomposición de fecha y segundos del tiempo de registro.")
    ap.add_argument("--input", "-i", required=True, help="Ruta del CSV de entrada.")
    ap.add_argument("--output", "-o", required=True, help="Ruta del CSV de salida.")
    ap.add_argument("--encoding", default="utf-8-sig", help="Encoding del CSV (por defecto utf-8-sig).")
    ap.add_argument("--sep", default=",", help="Separador del CSV (por defecto ,).")
    args = ap.parse_args()

    # Nombres exactos de columnas esperadas:
    COL_ID = "Id"
    COL_FECHA = "Fecha y hora"
    COL_POS = "posición"
    COL_DIST = "Distancia (m)"
    COL_TREG = "Tiempo de registro (s)"
    COL_VEL = "Velocidad (km/h)"

    try:
        df = pd.read_csv(args.input, encoding=args.encoding, sep=args.sep)
    except Exception as e:
        print(f"[ERROR] No pude leer el CSV: {e}", file=sys.stderr)
        sys.exit(1)

    # Validación mínima de columnas
    expected = {COL_ID, COL_FECHA, COL_POS, COL_DIST, COL_TREG, COL_VEL}
    missing = expected - set(df.columns)
    if missing:
        print(f"[ERROR] Faltan columnas en el CSV: {missing}", file=sys.stderr)
        sys.exit(1)

    # ---- Conversión de posición a lat/lon ----
    latitudes = []
    longitudes = []
    for pos in df[COL_POS]:
        lat, lon = pos_sw_to_latlon(pos)
        latitudes.append(lat)
        longitudes.append(lon)
    df["Latitud"]  = latitudes
    df["Longitud"] = longitudes

    # ---- Descomponer fecha y hora ----
    anos, meses, dias, horas, minutos, segundos = [], [], [], [], [], []
    for dt_str in df[COL_FECHA]:
        y, mo, d, h, mi, se = parse_datetime_peru(dt_str)
        anos.append(y); meses.append(mo); dias.append(d)
        horas.append(h); minutos.append(mi); segundos.append(se)
    df["Año"] = anos
    df["Mes"] = meses
    df["Día"] = dias
    df["Hora"] = horas
    df["Minuto"] = minutos
    df["Segundo"] = segundos

    # ---- Tiempo de registro a segundos ----
    df["TiempoRegistro_Seg"] = [hms_to_seconds(x) for x in df[COL_TREG]]

    # Orden sugerido de columnas (puedes ajustar):
    ordered_cols = [
        COL_ID,
        COL_FECHA, "Año", "Mes", "Día", "Hora", "Minuto", "Segundo",
        COL_POS, "Latitud", "Longitud",
        COL_DIST, COL_TREG, "TiempoRegistro_Seg",
        COL_VEL
    ]
    # Mantener también cualquier otra columna extra (si existiera)
    tail_cols = [c for c in df.columns if c not in ordered_cols]
    df = df[ordered_cols + tail_cols]

    # Guardar
    try:
        df.to_csv(args.output, index=False, encoding="utf-8-sig")
    except Exception as e:
        print(f"[ERROR] No pude escribir el CSV de salida: {e}", file=sys.stderr)
        sys.exit(1)

    print(f"[OK] Archivo generado: {args.output}")
    # Nota: si ves lat/long = NaN en algunas filas, revisa el formato exacto de 'posición'.

if __name__ == "__main__":
    main()
